## Text Extraction

In [39]:
import os

# os.chdir("Final")
print(os.getcwd()) 
print(os.path.exists("a.pdf"))

C:\Users\ashwinir\OneDrive - Jafferjee Brothers\Desktop\newGit\AIOps-Framework\Research_Manith\Advance\Final
True


In [42]:
!pip install pymupdf -qU


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('a.pdf')
documents = loader.load()
print(documents[0].page_content)

Annual Report
2023
Sri Lanka Telecom PLC


In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(documents)
print(chunks[0:4])

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.0 (Windows)', 'creationdate': '2024-04-18T08:47:52+05:30', 'source': 'a.pdf', 'file_path': 'a.pdf', 'total_pages': 204, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-18T11:46:47+05:30', 'trapped': '', 'modDate': "D:20240418114647+05'30'", 'creationDate': "D:20240418084752+05'30'", 'page': 0}, page_content='Annual Report\n2023\nSri Lanka Telecom PLC'), Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 16.0 (Windows)', 'creationdate': '2024-04-18T08:47:52+05:30', 'source': 'a.pdf', 'file_path': 'a.pdf', 'total_pages': 204, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-04-18T11:46:47+05:30', 'trapped': '', 'modDate': "D:20240418114647+05'30'", 'creationDate': "D:20240418084752+05'30'", 'page': 2}, page_content='We are an “App” of Life\nThe SLT-MOBITEL proposition is o

In [55]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["text"],
    template="Extract the names and dates from the following text:\n\n{text}"
)

llm = ChatOpenAI()
chain = LLMChain(llm=llm, prompt=prompt)

result = chain.run("John Doe was born on Jan 1st, 1990.")
print(result)

ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'nam...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [56]:
from langchain.output_parsers import RegexParser

parser = RegexParser(
    regex=r"Name: (.*), Date: (.*)",
    output_keys=["name", "date"]
)
text = "Name: Alice, Date: 2023-01-01"
parsed = parser.parse(text)

In [57]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

db = FAISS.from_documents(chunks, OpenAIEmbeddings())
results = db.similarity_search("Who is Alice?")

C:\Users\ashwinir\AppData\Local\Temp\ipykernel_12872\2390252827.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(chunks, OpenAIEmbeddings())


ValidationError: 1 validation error for OpenAIEmbeddings
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'cli...20, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), retriever=db.as_retriever())
response = qa.run("When was the document created?")

In [ ]:
from langchain.schema.runnable import RunnableLambda

pipeline = loader | splitter | RunnableLambda(print)
pipeline.invoke("example.pdf")